In [ ]:
import datajoint as dj
import sys
sys.path.append('/Users/carlos/Github/UberPhys/src')

import numpy as np

import sys
import pickle
# sys.path.append('/Users/carlos/Github/UberPhys/src')
sys.path.append('/Users/carlos/Github/bl_pipeline_python')
import datajoint as dj
import utility.blob_transformation as bt
import os


dj.blob.use_32bit_dims = True  # necessary for pd.blob read

dj.config['database.host'] = 'datajoint01.pni.princeton.edu'
dj.config['database.host'] = 'bdata00.pni.princeton.edu'
dj.config['database.user'] = 'brody'  # enter your puid username

dj.conn()

if dj.config['database.host'] != 'bdata00.pni.princeton.edu':
    raise ValueError("Only run this on bdata00.pni.princeton.edu")


In [ ]:
bdata.Sessions()

In [ ]:
bdata = dj.create_virtual_module('bdata', 'bdata')

ratname = "A327"
savedir = "/Users/carlos/Github/bl_pipeline_python/carlos"

In [ ]:
sids = (bdata.Sessions & "ratname='%s'" % ratname).fetch('sessid')  # session ids for this rat

sessionsAndBevs = []
for i in range(sids.size):
    sid = sids[i]  # a single session id
    parsed_events = (bdata.ParsedEvents & "sessid=%d" % sid).fetch(as_dict=True)  # get the parsed events for this session
    # Now put them through Alvaro's pipeline:
    peh = bt.transform_blob(parsed_events[0]['peh'])
    df_trial_peh = bt.blob_peh_to_df(peh, append_original_columnname=True)
    df_event = bt.peh_trial_df_to_event_df(df_trial_peh, sid)

    # Finally, append the session's events to the list of sessionsAndBevs
    sessionsAndBevs.append(df_event.to_dict('records'))
    if i % 10 == 0:
        print("Processed %d/%d sessions" % (i, sids.size))

# save the pickle file:
with open("%s/%s_sessionsAndBevs.pkl" % (savedir, ratname), 'wb') as f:
    pickle.dump(sessionsAndBevs, f)
    print("Saved %s/%s_sessionsAndBevs.pkl" % (savedir, ratname))



## Sandbox

In [ ]:
bdata.BehaviorEvent() & "sessid = 903003"